In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch import nn
import pandas as pd
from timeit import default_timer as timer
import models.trainer_lib as tl
import models.torch_model_definitions as tmd
import os
from configs import CONFIGS, load_data
import random
import json

with open('final_eval_results/strategies.json', 'r') as f:
    strategies = json.load(f)

available_strategies = [s for s in strategies if 'fold_9' in strategies[s]]
if len(available_strategies) == 0:
    raise Exception('No strategies available, make sure to run eval_final_config.py with -sm switch or download final_eval_results folder from GitHub repo')

print('Strategies available:', ', '.join(available_strategies))

# Choose strategy and fold, Initialization cell

In [ ]:
FOLD = 3
CHOICE = "mimo_tcn"

### Above are configurable parameters ###

if CHOICE not in available_strategies:
    raise Exception(f'Strategy {CHOICE} not available, choose one: {", ".join(available_strategies)}')
if FOLD not in range(1, 10):
    raise Exception(f'Fold {FOLD} not available, choose one from 1-9')

config = CONFIGS[CHOICE]
path = f"final_eval_results/{CHOICE}/fold_{FOLD}.pt"
wrapper = config['wrapper'](config['model'](**config['model_params']), config['seq_len'], config['pred_len'])

wrapper.load_state(path)
X, y = load_data(config['load_modifier'])

split_len = X.shape[0] // (config['n_splits'] + 1) 
start_point = split_len * FOLD
end_point = split_len * (FOLD + 1)

y_pred, y_true = wrapper.predict(X[start_point:end_point], y[start_point:end_point])

## Per hour ahead performance

In [ ]:
df = pd.read_csv(f'final_eval_results/{CHOICE}.csv')

df_no12fold = df[(df['Fold'] != 1) & (df['Fold'] != 2)]
      
means = df_no12fold.drop(['Fold', 'Train Time', 'Pred Time'], axis=1).groupby('Hour').mean()
stds = df_no12fold.drop(['Fold', 'Train Time', 'Pred Time'], axis=1).groupby('Hour').std()

means = means.round({'MAE': 2, 'MSE': 1, 'RMSE': 2, 'MAPE': 4, 'MPE': 4})
stds = stds.round({'MAE': 2, 'MSE': 1, 'RMSE': 2, 'MAPE': 4, 'MPE': 4})

mean_and_std = pd.DataFrame(columns=['Hour', 'MAE', 'MSE', 'RMSE', 'MAPE', 'MPE'])
mean_and_std['Hour'] = means.index
mean_and_std.set_index('Hour', inplace=True)

for col in means.columns:
    mean_and_std[col] = means[col].astype(str) + "\u00B1" + stds[col].astype(str)

# print(mean_and_std.to_latex())
# print(mean_and_std.to_markdown())

mean_and_std

## Per fold performance

In [ ]:
df = pd.read_csv(f'final_eval_results/{CHOICE}.csv')

df = df[(df['Hour'] == 0)]

means = df.drop(['Hour', 'Train Time', 'Pred Time'], axis=1).groupby('Fold').mean()
stds = df.drop(['Hour', 'Train Time', 'Pred Time'], axis=1).groupby('Fold').std()

means = means.round({'MAE': 2, 'MSE': 1, 'RMSE': 2, 'MAPE': 4, 'MPE': 4})
stds = stds.round({'MAE': 2, 'MSE': 1, 'RMSE': 2, 'MAPE': 4, 'MPE': 4})

mean_and_std = pd.DataFrame(columns=['Fold', 'MAE', 'MSE', 'RMSE', 'MAPE', 'MPE'])
mean_and_std.set_index('Fold', inplace=True)

for col in means.columns:
    mean_and_std[col] = means[col].astype(str) + "\u00B1" + stds[col].astype(str)
    
mean_and_std

## Training and prediction time

In [ ]:
df = pd.read_csv(f'final_eval_results/{CHOICE}.csv')

print(f"Train time: {df['Train Time'].mean():.1f}\u00B1{df['Train Time'].std():.1f}")
print(f"Pred time: {df['Pred Time'].mean():.4f}\u00B1{df['Pred Time'].std():.4f}")

tt_df = df.groupby('Fold')['Train Time'].mean()
tt_df = tt_df / 60

pt_df = df.groupby('Fold')['Pred Time'].mean()
pt_df = pt_df * 1000

fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(15, 6))
axs[0].bar(tt_df.index, tt_df, color='orange')
axs[0].set_title("Training Time")
axs[0].set_xlabel("Fold")
axs[0].set_ylabel("Time (minutes)")
axs[0].grid(axis='y')

axs[1].bar(pt_df.index, pt_df, color='blue')
axs[1].set_title("Prediction Time")
axs[1].set_xlabel("Fold")
axs[1].set_ylabel("Time (ms)")
axs[1].grid(axis='y')

## Prediction plots

In [ ]:
PLOT_START = None # None means choose randomly
PLOT_LENGTH = 150
WHICH_PLOTS = [ # Comment out the ones you don't want
#    'per_hour',
    'together',
]

### Above are configurable parameters ###

if PLOT_START is None:
    PLOT_START = random.randint(0, split_len - PLOT_LENGTH)
    
if PLOT_START + PLOT_LENGTH > split_len:
    raise ValueError(f'PLOT_START + PLOT_LENGTH > split_len, PLOT_START: {PLOT_START}, PLOT_LENGTH: {PLOT_LENGTH}, split_len: {split_len}')
    
if 'per_hour' in WHICH_PLOTS:
    wrapper.plot_predictions_per_hour(y_pred[PLOT_START:PLOT_START+PLOT_LENGTH], y_true[PLOT_START:PLOT_START+PLOT_LENGTH])
if 'together' in WHICH_PLOTS:
    wrapper.plot_predictions_together(y_pred[PLOT_START:PLOT_START+PLOT_LENGTH], y_true[PLOT_START:PLOT_START+PLOT_LENGTH])